In [102]:
from sqlalchemy import create_engine
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from env_vars import *
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

In [103]:
engine = create_engine('sqlite:///song_list.db', echo=False)

In [104]:
song_list = pd.read_pickle("./final_data/DF_v1.pkl")

In [37]:
song_list.head()

,acousticness,artist,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,songid,speechiness,tempo,time_signature,track,valence
0,0.001300,Karkkiautomaatti,0.487,157307.0,0.678,0.0551,9.0,0.0846,-7.780,1.0,5PS5dpaLogPzYU9hWiWyZb,0.0495,149.940,4.0,Tanssi vaan,0.870
1,0.000045,Hudson Mohawke,0.662,138960.0,0.823,0.9520,4.0,0.3430,-1.711,0.0,41RpZW2lxAdnqDd2nMBzLQ,0.0662,177.745,4.0,No One Could Ever,0.621
2,0.000725,Der Mystic,0.430,514290.0,0.960,0.9250,5.0,0.1230,-7.741,0.0,2poHURuOfVNbzZdivAwtOH,0.0431,140.001,4.0,Tangle Of Aspens,0.225
3,0.245000,Sasha / Turbulence,0.663,212413.0,0.677,0.0000,3.0,0.3500,-4.117,0.0,1jg9hZnReygpBvV2axGuPy,0.0755,152.085,4.0,We Have Got Love,0.879
4,0.002760,Kris Kross,0.859,221200.0,0.741,0.0000,11.0,0.3250,-12.329,0.0,3GsS8jzoixpCnp4jDWCEvb,0.2710,98.082,4.0,2 Da Beat Ch'yall,0.529


In [106]:
song_list.to_sql('features', con=engine,if_exists='replace')

In [113]:
def authorize():
    client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Inserts Genre Value into Genres Column

In [ ]:
genres_list = []
for item in engine.execute("SELECT songid FROM features").fetchall()[11325:]:    
    songid = item.values()[0]
    artistid = get_artist_id(songid)
    genres = get_genres(artistid)
    try:
        engine.execute(f'UPDATE features SET genres = \"{genres}\" WHERE songid = \"{songid}\"')
        authorize()
    except:
        print("couldn't insert")
        continue

couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't insert
couldn't

In [87]:
# null = []
# for genre in genres_list:
#     if genre == []:
#         null.append(genre)

In [29]:
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [51]:
def get_features(trackID):
        return sp.audio_features(trackID)[0]
    
get_features('54flyrjcdnQdco7300avMJ')

{'danceability': 0.693,
 'energy': 0.497,
 'key': 2,
 'loudness': -7.316,
 'mode': 1,
 'speechiness': 0.119,
 'acousticness': 0.679,
 'instrumentalness': 0,
 'liveness': 0.258,
 'valence': 0.473,
 'tempo': 81.308,
 'type': 'audio_features',
 'id': '54flyrjcdnQdco7300avMJ',
 'uri': 'spotify:track:54flyrjcdnQdco7300avMJ',
 'track_href': 'https://api.spotify.com/v1/tracks/54flyrjcdnQdco7300avMJ',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/54flyrjcdnQdco7300avMJ',
 'duration_ms': 122067,
 'time_signature': 4}

In [24]:
def get_artist_id(trackID):
    return sp.track(trackID)['artists'][0]['id']
    


get_artist_id('54flyrjcdnQdco7300avMJ')

'1dfeR4HaWDbWqFHLkxsg1d'

In [26]:
def get_genres(artistID):
    return sp.artist(artistID)['genres']


get_genres('1dfeR4HaWDbWqFHLkxsg1d')

['glam rock', 'rock']

In [ ]:
import sqlite3

def create_table(name):
    conn = sqlite3.connect(f'./{name}.sqlite3')
    curs = conn.cursor()
    create_table = 'CREATE TABLE File (songid varchar(1000),artist varchar(1000),track varchar(1000), danceability float, energy float, key float, loudness float, mode float, speechiness float, acousticness float, instrumentalness float, liveness float, valence float, tempo float, duration_ms float,time_signature float);' 
    curs.execute(create_table)
    return conn

def insert(conn,f):    
    curs = conn.cursor()    
    query = (f'INSERT INTO File (songid, artist, track, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_ms,time_signature) VALUES (\"{f["songid"]}\",\"{f["artist"]}\",\"{f["track"]}\", {f["danceability"]}, {f["energy"]}, {f["key"]}, {f["loudness"]}, {f["mode"]}, {f["speechiness"]}, {f["acousticness"]},  {f["instrumentalness"]}, {f["liveness"]}, {f["valence"]}, {f["tempo"]}, {f["duration_ms"]},{f["time_signature"]});')
    curs.execute(query)
    curs.close()
    conn.commit()

In [ ]:
def retrieve_genres():
    